<a href="https://colab.research.google.com/github/sebamelies/Olist/blob/main/AEI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INSTALL

In [ ]:
!pip install fuzzywuzzy
!pip install unicode
!pip install deep_translator
!pip install googletrans

# Manipulación y análisis de datos
import pandas as pd
import numpy as np

# Visualización de datos
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

# Expresiones regulares
import re
from fuzzywuzzy import process

import warnings
warnings.filterwarnings('ignore')

from deep_translator import GoogleTranslator

from deep_translator import GoogleTranslator
import concurrent.futures

from googletrans import Translator
import concurrent.futures

import logging
import missingno as msno

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# FUNCIONES

Check df

In [ ]:
def check_df(df, tipo):
  # tipo == 'simple' - Solo muestra volumenes y cabecera
  if tipo == 'simple':
    print("¿Cuántas filas y columnas hay en el conjunto de datos?")
    num_filas, num_columnas = df.shape
    print("\tHay {:,} filas y {:,} columnas.".format(num_filas, num_columnas))
    print('\n########################################################################################')

    print("¿Cuáles son las primeras dos filas del conjunto de datos?")
    display(df.head(2))
    print('\n########################################################################################')

  else:
    print("¿Cuántas filas y columnas hay en el conjunto de datos?")
    num_filas, num_columnas = df.shape
    print("\tHay {:,} filas y {:,} columnas.".format(num_filas, num_columnas))
    print('\n########################################################################################')

    print("¿Cuáles son las primeras cinco filas del conjunto de datos?")
    display(df.head(5))
    print('\n########################################################################################')

    print("¿Cuáles son las últimas cinco filas del conjunto de datos?")
    display(df.tail(5))
    print('\n########################################################################################')

    print("¿Cómo puedes obtener una muestra aleatoria de filas del conjunto de datos?")
    display(df.sample(5))
    print('\n########################################################################################')

    print("¿Cuáles son las columnas del conjunto de datos?")
    for i in list(df.columns):
      print('\t - ' + i)
    print('\n########################################################################################')

    print("¿Cuál es el tipo de datos de cada columna?")
    print(df.dtypes)
    print('\n########################################################################################')

    print("¿Cuántas columnas hay de cada tipo de datos?")
    print(df.dtypes.value_counts())
    print('\n########################################################################################')

    print("¿Cómo podríamos obtener información más completa sobre la estructura y el contenido del DataFrame?")
    print(df.info())
    print('\n########################################################################################')

    print("¿Cuántos valores únicos tiene cada columna?")
    print(df.nunique())
    print('\n########################################################################################')

    print("¿Cuáles son las estadísticas descriptivas básicas de las columnas numéricas?")
    display(df.describe(include = 'all').fillna(''))
    print('\n########################################################################################')

    print("¿Hay valores nulos en el conjunto de datos?")
    print(df.isnull().sum().sort_values(ascending = False))
    print('\n########################################################################################')

    print("¿Cuál es el porcentaje de valores nulos en cada columna?")
    print(round((df.isnull().sum()/len(df)*100), 2).sort_values(ascending = False))
    print('\n########################################################################################')

Detección de outliers

In [ ]:
def deteccion_outliers (df, variable):
    # Suponiendo que tienes un DataFrame df y quieres analizar la columna 'columna_de_interes'
    columna = df[variable]

    sns.boxplot(
      data=df,
      y=variable,
    )
    plt.show()

    Q1 = columna.quantile(0.25)
    Q3 = columna.quantile(0.75)
    IQR = Q3 - Q1

    print('Valor del segundo cuartil (25%): {:.2f}'.format(Q1))
    print('Valor del tercer cuartil (75%): {:.2f}'.format(Q3))
    print('Valor del rango intercuartil (IQR): {:.2f}'.format(IQR))

    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR

    print(f"Los valores atípicos se definen como aquellos que caen fuera del siguiente rango:")
    print(f"\t - Límite inferior (considerado extremadamente bajo): {limite_inferior:.2f}")
    print(f"\t - Límite superior (considerado extremadamente alto): {limite_superior:.2f}")


    outilers = list(columna[((columna < limite_inferior) | (columna > limite_superior))].index)
    num_outliers = len(outilers)
    print(f"Hay {num_outliers} outliers en la variable '{variable}'")
    return outilers

Procesar fechas

In [ ]:
def procesar_fecha(fecha):
  '''
    * Separados por "-":
      - Patrón 1: 04-01-2020
      - Patrón 2: 2020-01-10
      - Patrón 3: 01-14-20

    * Separados por "/":
      - Patrón 4: 11/01/2020
      - Patrón 5: 02/03/20
  '''

  # Separador '-'

  # %d-%m-%y'
  patron1 = r'^(0[1-9]|[12][0-9]|3[01])-(0[1-9]|1[0-2])-(\d{2})$'
  # dia: (0[1-9]|[12][0-9]|3[01])
  # mes: (0[1-9]|1[0-2])
  # año: (\d{2})

  #'%d-%m-%Y'
  patron2 = r'^(0[1-9]|[12][0-9]|3[01])-(0[1-9]|1[0-2])-(\d{4})$'

  #'%m-%d-%y'
  patron3 = r'^(0[1-9]|1[0-2])-(0[1-9]|[12][0-9]|3[01])-(\d{2})$'

  #'%m-%d-%Y'
  patron4 = r'^(0[1-9]|1[0-2])-(0[1-9]|[12][0-9]|3[01])-(\d{4})$'

  #'%Y-%m-%d'
  patron5 = r'^(\d{4})-(0[1-9]|1[0-2])-(0[1-9]|[12][0-9]|3[01])$'

  # Separador '/'

  #'%d/%m/%y'
  patron6 = r'^(0[1-9]|[12][0-9]|3[01])/(0[1-9]|1[0-2])/(\d{2})$'

  #'%m/%d/%y'
  patron7 = r'^(0[1-9]|1[0-2])/(0[1-9]|[12][0-9]|3[01])/(\d{2})$'

  #'%m/%d/%Y'
  patron8 = r'^(0[1-9]|1[0-2])/(0[1-9]|[12][0-9]|3[01])/(\d{4})$'

  #'%Y/%m/%d'
  patron9 = r'^(\d{4})/(0[1-9]|1[0-2])/(0[1-9]|[12][0-9]|3[01])$'

  #'%Y/%m/%d'
  patron10 = r'^(0[1-9]|[12][0-9]|3[01])/(0[1-9]|1[0-2])/(\d{4})$'

  # 12/5/2021
  #'%Y/%m/%d'
  patron11 = r'^(0?[1-9]|1[0-2])/(0?[1-9]|[12][0-9]|3[01])/(\d{4})$'

  # Comprueba si la fecha cumple con el patrón
  if pd.notnull(fecha) and re.fullmatch(patron1, fecha):
    # Parsea la fecha al formato deseado y devuelve en formato "aaaa-mm-dd"
    return pd.to_datetime(fecha, format='%d-%m-%y').strftime('%Y-%m-%d')

  elif pd.notnull(fecha) and re.fullmatch(patron2, fecha):
    return pd.to_datetime(fecha, format='%d-%m-%Y').strftime('%Y-%m-%d')

  elif pd.notnull(fecha) and re.fullmatch(patron3, fecha):
    return pd.to_datetime(fecha, format='%m-%d-%y').strftime('%Y-%m-%d')

  elif pd.notnull(fecha) and re.fullmatch(patron4, fecha):
    return pd.to_datetime(fecha, format='%m-%d-%Y').strftime('%Y-%m-%d')

  elif pd.notnull(fecha) and re.fullmatch(patron5, fecha):
    return pd.to_datetime(fecha, format='%Y-%m-%d').strftime('%Y-%m-%d')

  elif pd.notnull(fecha) and re.fullmatch(patron6, fecha):
    return pd.to_datetime(fecha, format='%d/%m/%y').strftime('%Y-%m-%d')

  elif pd.notnull(fecha) and re.fullmatch(patron7, fecha):
      return pd.to_datetime(fecha, format='%m/%d/%y').strftime('%Y-%m-%d')

  elif pd.notnull(fecha) and re.fullmatch(patron8, fecha):
      return pd.to_datetime(fecha, format='%m/%d/%Y').strftime('%Y-%m-%d')

  elif pd.notnull(fecha) and re.fullmatch(patron9, fecha):
      return pd.to_datetime(fecha, format='%Y/%m/%d').strftime('%Y-%m-%d')

  elif pd.notnull(fecha) and re.fullmatch(patron10, fecha):
      return pd.to_datetime(fecha, format='%d/%m/%Y').strftime('%Y-%m-%d')

  # 12/5/2021
  elif pd.notnull(fecha) and re.fullmatch(patron11, fecha):
      return pd.to_datetime(fecha, format='%m/%d/%Y').strftime('%Y-%m-%d')

  else:
      # Devuelve la fecha original si no cumple con el patrón o es NaN
      return pd.NaT  # Retorna Not a Time para fechas que no coinciden con ningún formato


# OLIST CUSTOMERS

In [ ]:
df_customers_with_geolocation = pd.read_parquet('/content//drive/MyDrive/e-commerce TFM/Data/df_customer_with_geolocation.parquet')


In [ ]:
check_df(df=df_customers_with_geolocation, tipo="")

¿Cuántas filas y columnas hay en el conjunto de datos?
	Hay 99,441 filas y 9 columnas.

########################################################################################
¿Cuáles son las primeras cinco filas del conjunto de datos?


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,customer_city,country
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,SP,14409,-20.499273,-47.396658,Franca,Brazil
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,09790,SP,09790,-23.728495,-46.542128,Sao Bernardo do Campo,Brazil
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,01151,SP,01151,-23.531143,-46.656816,Sao Paulo,Brazil
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,08775,SP,08775,-23.500670,-46.186348,Mogi das Cruzes,Brazil
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,SP,13056,-22.975708,-47.143140,Campinas,Brazil



########################################################################################
¿Cuáles son las últimas cinco filas del conjunto de datos?


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,customer_city,country
99436,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,03937,SP,03937,-23.586244,-46.499542,Sao Paulo,Brazil
99437,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,06764,SP,06764,-23.616873,-46.767500,Taboao da Serra,Brazil
99438,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,CE,60115,-3.735628,-38.510775,Fortaleza,Brazil
99439,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,RS,92120,-29.949964,-51.168380,Canoas,Brazil
99440,274fa6071e5e17fe303b9748641082c8,84732c5050c01db9b23e19ba39899398,06703,SP,06703,-23.596649,-46.911766,Cotia,Brazil



########################################################################################
¿Cómo puedes obtener una muestra aleatoria de filas del conjunto de datos?


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,customer_city,country
31620,61e6da519b71bef0cff6229f20cf8308,171b5c03529625a961df048b48c718b9,12945,SP,12945,-23.113261,-46.526972,Atibaia,Brazil
37107,ddb27d17205431758c18e24ce71f837f,a0aa8b33d6af1f31f86b91cb71e838f2,41120,BA,41120,-12.970210,-38.462745,Salvador,Brazil
16143,e372e544ed515524e099e4f543d9aeb3,b067f83c11f3d3deda01a9e220ac0aa2,11015,SP,11015,-23.953172,-46.318865,Santos,Brazil
16272,d15b4822b0f37b8463bfebc5c82608ef,bd3d8576417686aaa8c3973f7b5a0ba3,38061,MG,38061,-19.744847,-47.954715,Uberaba,Brazil
5773,e8479a6bb05afc8b19f0c49788d9003f,87ce5e73c2b5b54afa807fcc53fef189,05075,SP,05075,-23.522363,-46.708559,Sao Paulo,Brazil



########################################################################################
¿Cuáles son las columnas del conjunto de datos?
	 - customer_id
	 - customer_unique_id
	 - customer_zip_code_prefix
	 - customer_state
	 - geolocation_zip_code_prefix
	 - geolocation_lat
	 - geolocation_lng
	 - customer_city
	 - country

########################################################################################
¿Cuál es el tipo de datos de cada columna?
customer_id                     object
customer_unique_id              object
customer_zip_code_prefix        object
customer_state                  object
geolocation_zip_code_prefix     object
geolocation_lat                float64
geolocation_lng                float64
customer_city                   object
country                         object
dtype: object

########################################################################################
¿Cuántas columnas hay de cada tipo de datos?
object     7
float64    2
Name: count, d

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,customer_city,country
count,99441,99441,99441,99441,99163,99163.0,99163.0,99441,99441
unique,99441,96096,14994,27,14837,,,4119,1
top,06b8999e2fba1a1fbc88172c00ba8bc7,8d50f5eadf50201ccdcedfb9e2ac8455,22790,SP,22790,,,Sao Paulo,Brazil
freq,1,17,142,41746,142,,,15540,99441
mean,,,,,,-21.190867,-46.175531,,
std,,,,,,5.609024,4.056752,,
min,,,,,,-33.689824,-72.668821,,
25%,,,,,,-23.589332,-48.097915,,
50%,,,,,,-22.924854,-46.6304,,
75%,,,,,,-20.139309,-43.597342,,



########################################################################################
¿Hay valores nulos en el conjunto de datos?
geolocation_zip_code_prefix    278
geolocation_lat                278
geolocation_lng                278
customer_id                      0
customer_unique_id               0
customer_zip_code_prefix         0
customer_state                   0
customer_city                    0
country                          0
dtype: int64

########################################################################################
¿Cuál es el porcentaje de valores nulos en cada columna?
geolocation_zip_code_prefix    0.28
geolocation_lat                0.28
geolocation_lng                0.28
customer_id                    0.00
customer_unique_id             0.00
customer_zip_code_prefix       0.00
customer_state                 0.00
customer_city                  0.00
country                        0.00
dtype: float64

###################################################

In [ ]:
df_customers_with_geolocation['customer_id'].nunique()

99441

# OLIST GEOLOCATION

In [ ]:
df_geolocation = pd.read_parquet('/content//drive/MyDrive/e-commerce TFM/Data/df_geolocation_clean.parquet')


In [ ]:
check_df(df=df_geolocation, tipo="")


¿Cuántas filas y columnas hay en el conjunto de datos?
	Hay 19,015 filas y 5 columnas.

########################################################################################
¿Cuáles son las primeras cinco filas del conjunto de datos?


,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_state,geolocation_city
0,01001,-23.550271,-46.634047,SP,Sao Paulo
1,01002,-23.547657,-46.634991,SP,Sao Paulo
2,01003,-23.548991,-46.635653,SP,Sao Paulo
3,01004,-23.549829,-46.634792,SP,Sao Paulo
4,01005,-23.549487,-46.636650,SP,Sao Paulo



########################################################################################
¿Cuáles son las últimas cinco filas del conjunto de datos?


,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_state,geolocation_city
19010,99960,-27.953722,-52.025511,RS,Charrua
19011,99965,-28.183372,-52.039850,RS,Agua Santa
19012,99970,-28.343920,-51.874988,RS,Ciriaco
19013,99980,-28.389500,-51.843122,RS,David Canabarro
19014,99990,-28.329595,-51.769362,RS,Muliterno



########################################################################################
¿Cómo puedes obtener una muestra aleatoria de filas del conjunto de datos?


,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_state,geolocation_city
8363,32653,-19.950263,-44.169979,MG,Betim
10947,49055,-10.912969,-37.061812,SE,Aracaju
10451,45078,-14.838323,-40.875991,BA,Vitoria da Conquista
3031,05867,-23.673019,-46.769992,SP,Sao Paulo
7204,26020,-22.732334,-43.449818,RJ,Nova Iguacu



########################################################################################
¿Cuáles son las columnas del conjunto de datos?
	 - geolocation_zip_code_prefix
	 - geolocation_lat
	 - geolocation_lng
	 - geolocation_state
	 - geolocation_city

########################################################################################
¿Cuál es el tipo de datos de cada columna?
geolocation_zip_code_prefix     object
geolocation_lat                float64
geolocation_lng                float64
geolocation_state               object
geolocation_city                object
dtype: object

########################################################################################
¿Cuántas columnas hay de cada tipo de datos?
object     3
float64    2
Name: count, dtype: int64

########################################################################################
¿Cómo podríamos obtener información más completa sobre la estructura y el contenido del DataFrame?
<class 'pandas.core.frame.Dat

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_state,geolocation_city
count,19015,19015.0,19015.0,19015,19015
unique,19015,,,27,5771
top,01001,,,SP,Sao Paulo
freq,1,,,6349,3180
mean,,-19.054528,-46.057051,,
std,,7.29705,5.367748,,
min,,-33.689824,-72.916069,,
25%,,-23.564158,-49.002485,,
50%,,-22.418164,-46.632063,,
75%,,-15.608744,-43.255602,,



########################################################################################
¿Hay valores nulos en el conjunto de datos?
geolocation_zip_code_prefix    0
geolocation_lat                0
geolocation_lng                0
geolocation_state              0
geolocation_city               0
dtype: int64

########################################################################################
¿Cuál es el porcentaje de valores nulos en cada columna?
geolocation_zip_code_prefix    0.0
geolocation_lat                0.0
geolocation_lng                0.0
geolocation_state              0.0
geolocation_city               0.0
dtype: float64

########################################################################################


# OLIST ORDER ITEMS

In [ ]:
df_order_items = pd.read_parquet('/content//drive/MyDrive/e-commerce TFM/Data/df_order_items.parquet')

In [ ]:
check_df(df=df_order_items, tipo="")


¿Cuántas filas y columnas hay en el conjunto de datos?
	Hay 112,650 filas y 7 columnas.

########################################################################################
¿Cuáles son las primeras cinco filas del conjunto de datos?


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14



########################################################################################
¿Cuáles son las últimas cinco filas del conjunto de datos?


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72
112649,fffe41c64501cc87c801fd61db3f6244,1,350688d9dc1e75ff97be326363655e01,f7ccf836d21b2fb1de37564105216cc1,2018-06-12 17:10:13,43.00,12.79



########################################################################################
¿Cómo puedes obtener una muestra aleatoria de filas del conjunto de datos?


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
79953,b5d8e4c3414692eee1f0a38254252823,1,6c09be1fc42a113b2f3e558d7df2f133,671585f5a2af58b6e276bc01003c0d2b,2018-08-21 19:30:13,45.00,17.25
76729,aead5f6413ef9943082b3d3cfd37ccbe,3,610ab592e0d325dee460ae2c00f1a85c,077c5fae4bea9500e3737b16f71b9d3a,2018-06-28 01:30:32,28.80,11.93
50011,71ab839e1fe570014f0654f3e929b7cb,1,22c5ddc22ea8cae84fc9917def682dc6,cc419e0650a3c5ba77189a1882b7556a,2018-01-25 03:51:14,56.99,8.72
12513,1c58c266f70494954b0e545444355f3b,1,e44f675b60b3a3a2453ec36421e06f0f,218d46b86c1881d022bce9c68a7d4b15,2018-03-02 08:50:23,107.00,15.50
49103,6f893071ef9c8c6854e6e83921d2135e,1,db753afa921ad9943950ac46194a0031,430315b7bb4b6e4b3c978f9dfa9b0558,2018-01-30 20:14:22,25.00,25.63



########################################################################################
¿Cuáles son las columnas del conjunto de datos?
	 - order_id
	 - order_item_id
	 - product_id
	 - seller_id
	 - shipping_limit_date
	 - price
	 - freight_value

########################################################################################
¿Cuál es el tipo de datos de cada columna?
order_id                object
order_item_id            int64
product_id              object
seller_id               object
shipping_limit_date     object
price                  float64
freight_value          float64
dtype: object

########################################################################################
¿Cuántas columnas hay de cada tipo de datos?
object     4
float64    2
int64      1
Name: count, dtype: int64

########################################################################################
¿Cómo podríamos obtener información más completa sobre la estructura y el contenido del DataFram

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
count,112650,112650.0,112650,112650,112650,112650.0,112650.0
unique,98666,,32951,3095,93318,,
top,8272b63d03f5f79c56e9e4120aec44ef,,aca2eb7d00ea1a7b8ebd4e68314663af,6560211a19b47992c3666cc44a7e94c0,2017-07-21 18:25:23,,
freq,21,,527,2033,21,,
mean,,1.197834,,,,120.653739,19.99032
std,,0.705124,,,,183.633928,15.806405
min,,1.0,,,,0.85,0.0
25%,,1.0,,,,39.9,13.08
50%,,1.0,,,,74.99,16.26
75%,,1.0,,,,134.9,21.15



########################################################################################
¿Hay valores nulos en el conjunto de datos?
order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64

########################################################################################
¿Cuál es el porcentaje de valores nulos en cada columna?
order_id               0.0
order_item_id          0.0
product_id             0.0
seller_id              0.0
shipping_limit_date    0.0
price                  0.0
freight_value          0.0
dtype: float64

########################################################################################


# OLIST ORDER PAYMENTS

In [ ]:
df_order_payments = pd.read_parquet('/content//drive/MyDrive/e-commerce TFM/Data/df_order_payments.parquet')


In [ ]:
check_df(df=df_order_payments, tipo="")


¿Cuántas filas y columnas hay en el conjunto de datos?
	Hay 103,886 filas y 5 columnas.

########################################################################################
¿Cuáles son las primeras cinco filas del conjunto de datos?


,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45



########################################################################################
¿Cuáles son las últimas cinco filas del conjunto de datos?


,order_id,payment_sequential,payment_type,payment_installments,payment_value
103881,0406037ad97740d563a178ecc7a2075c,1,boleto,1,363.31
103882,7b905861d7c825891d6347454ea7863f,1,credit_card,2,96.80
103883,32609bbb3dd69b3c066a6860554a77bf,1,credit_card,1,47.77
103884,b8b61059626efa996a60be9bb9320e10,1,credit_card,5,369.54
103885,28bbae6599b09d39ca406b747b6632b1,1,boleto,1,191.58



########################################################################################
¿Cómo puedes obtener una muestra aleatoria de filas del conjunto de datos?


,order_id,payment_sequential,payment_type,payment_installments,payment_value
21935,713646be0226c051688714f343a1cda5,1,credit_card,3,113.41
61542,ee24b5ab6e72c956f37630115a3c5cc5,1,credit_card,1,36.94
28271,8836e9da58524060994ead4403874ad3,1,boleto,1,75.17
75636,abbd44be67f3cad08ffac14daf36af60,1,credit_card,1,52.15
51029,1e2ecf33949aeb71c490a6670f311f13,1,credit_card,10,283.99



########################################################################################
¿Cuáles son las columnas del conjunto de datos?
	 - order_id
	 - payment_sequential
	 - payment_type
	 - payment_installments
	 - payment_value

########################################################################################
¿Cuál es el tipo de datos de cada columna?
order_id                 object
payment_sequential        int64
payment_type             object
payment_installments      int64
payment_value           float64
dtype: object

########################################################################################
¿Cuántas columnas hay de cada tipo de datos?
object     2
int64      2
float64    1
Name: count, dtype: int64

########################################################################################
¿Cómo podríamos obtener información más completa sobre la estructura y el contenido del DataFrame?
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to

,order_id,payment_sequential,payment_type,payment_installments,payment_value
count,103886,103886.0,103886,103886.0,103886.0
unique,99440,,5,,
top,fa65dad1b0e818e3ccc5cb0e39231352,,credit_card,,
freq,29,,76795,,
mean,,1.092679,,2.853349,154.10038
std,,0.706584,,2.687051,217.494064
min,,1.0,,0.0,0.0
25%,,1.0,,1.0,56.79
50%,,1.0,,1.0,100.0
75%,,1.0,,4.0,171.8375



########################################################################################
¿Hay valores nulos en el conjunto de datos?
order_id                0
payment_sequential      0
payment_type            0
payment_installments    0
payment_value           0
dtype: int64

########################################################################################
¿Cuál es el porcentaje de valores nulos en cada columna?
order_id                0.0
payment_sequential      0.0
payment_type            0.0
payment_installments    0.0
payment_value           0.0
dtype: float64

########################################################################################



# OLIST ORDER REVIEWS

In [ ]:
df_order_reviews = pd.read_parquet('/content//drive/MyDrive/e-commerce TFM/Data/df_order_reviews.parquet')


In [ ]:
df_translated_order_reviews = pd.read_parquet('/content//drive/MyDrive/e-commerce TFM/Data/df_translated_reviews.parquet')


In [ ]:
check_df(df=df_translated_order_reviews, tipo="")

¿Cuántas filas y columnas hay en el conjunto de datos?
	Hay 99,224 filas y 9 columnas.

########################################################################################
¿Cuáles son las primeras cinco filas del conjunto de datos?


,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,translated_review_comment_title,translated_review_comment_message
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,,,2018-01-18,2018-01-18 21:46:59,,
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,,,2018-03-10,2018-03-11 03:05:13,,
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,,,2018-02-17,2018-02-18 14:36:24,,
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,,Recebi bem antes do prazo estipulado.,2017-04-21,2017-04-21 22:02:06,,Lo recibí mucho antes del plazo estipulado.
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,,Parabéns lojas lannister adorei comprar pela I...,2018-03-01,2018-03-02 10:26:53,,Felicidades tiendas lannister; me encantó comp...



########################################################################################
¿Cuáles son las últimas cinco filas del conjunto de datos?


,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,translated_review_comment_title,translated_review_comment_message
99219,574ed12dd733e5fa530cfd4bbf39d7c9,2a8c23fee101d4d5662fa670396eb8da,5,,,2018-07-07,2018-07-14 17:18:30,,
99220,f3897127253a9592a73be9bdfdf4ed7a,22ec9f0669f784db00fa86d035cf8602,5,,,2017-12-09,2017-12-11 20:06:42,,
99221,b3de70c89b1510c4cd3d0649fd302472,55d4004744368f5571d1f590031933e4,5,,Excelente mochila; entrega super rápida. Super...,2018-03-22,2018-03-23 09:10:43,,Excelente mochila; entrega súper rápida. ¡Reco...
99222,1adeb9d84d72fe4e337617733eb85149,7725825d039fc1f0ceb7635e3f7d9206,4,,,2018-07-01,2018-07-02 12:59:13,,
99223,efe49f1d6f951dd88b51e6ccd4cc548f,90531360ecb1eec2a1fbb265a0db0508,1,,meu produto chegou e ja tenho que devolver; po...,2017-07-03,2017-07-03 21:01:49,,Llegó mi producto y ya tengo que devolverlo; p...



########################################################################################
¿Cómo puedes obtener una muestra aleatoria de filas del conjunto de datos?


,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,translated_review_comment_title,translated_review_comment_message
8208,03578908a8e4f3976f70d798321565de,2d75fff7b2cc9e8df5ddfcf9a7a11cec,5,,,2018-04-24,2018-04-27 10:39:02,,
40090,b135813140a37944401840468ff060fd,c9cdc25f8eedefdbf5ddf8323c013010,5,,,2017-07-29,2017-08-06 21:07:09,,
52032,da2bb1600ad54fda631999da1539b575,c92a96b2edf7f42ebf19837f97f6009b,4,,,2017-08-25,2017-08-26 23:11:25,,
81649,aad2cad572bfb242f557e0681bcd51e0,eba75422c6a2414d3cd8289123e64200,4,Muito boa,Com certeza comprarei outros produtos.,2018-08-08,2018-08-09 16:24:01,Muy buena,Definitivamente compraré otros productos.
34234,132dc971e0e2a2a14c41465fcd37f788,317e9689b222bc8a5b08dfcb0c91b515,4,,Totalmente satisfeito.,2017-08-12,2017-08-14 11:55:06,,Totalmente satisfecho.



########################################################################################
¿Cuáles son las columnas del conjunto de datos?
	 - review_id
	 - order_id
	 - review_score
	 - review_comment_title
	 - review_comment_message
	 - review_creation_date
	 - review_answer_timestamp
	 - translated_review_comment_title
	 - translated_review_comment_message

########################################################################################
¿Cuál es el tipo de datos de cada columna?
review_id                                    object
order_id                                     object
review_score                                  int64
review_comment_title                         object
review_comment_message                       object
review_creation_date                 datetime64[ns]
review_answer_timestamp              datetime64[ns]
translated_review_comment_title              object
translated_review_comment_message            object
dtype: object

#######################

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,translated_review_comment_title,translated_review_comment_message
count,99224,99224,99224.0,99224,99224,99224,99224,99224,99224
unique,98410,98673,,4528,36103,,,3485,34145
top,7b606b0d57b078384f0b58eac1d41d78,c88b1d1b157a9999ce368f218a407141,,,,,,,
freq,3,3,,87656,58247,,,87743,58392
mean,,,4.086421,,,2018-01-12 20:49:23.948238336,2018-01-16 00:23:56.977938688,,
min,,,1.0,,,2016-10-02 00:00:00,2016-10-07 18:32:28,,
25%,,,4.0,,,2017-09-23 00:00:00,2017-09-27 01:53:27.249999872,,
50%,,,5.0,,,2018-02-02 00:00:00,2018-02-04 22:41:47.500000,,
75%,,,5.0,,,2018-05-16 00:00:00,2018-05-20 12:11:21.500000,,
max,,,5.0,,,2018-08-31 00:00:00,2018-10-29 12:27:35,,



########################################################################################
¿Hay valores nulos en el conjunto de datos?
review_id                            0
order_id                             0
review_score                         0
review_comment_title                 0
review_comment_message               0
review_creation_date                 0
review_answer_timestamp              0
translated_review_comment_title      0
translated_review_comment_message    0
dtype: int64

########################################################################################
¿Cuál es el porcentaje de valores nulos en cada columna?
review_id                            0.0
order_id                             0.0
review_score                         0.0
review_comment_title                 0.0
review_comment_message               0.0
review_creation_date                 0.0
review_answer_timestamp              0.0
translated_review_comment_title      0.0
translated_review_comment_m

# OLIST ORDERS

In [ ]:
df_orders = pd.read_parquet('/content//drive/MyDrive/e-commerce TFM/Data/df_orders.parquet')


In [ ]:
check_df(df=df_orders, tipo="")


¿Cuántas filas y columnas hay en el conjunto de datos?
	Hay 98,666 filas y 8 columnas.

########################################################################################
¿Cuáles son las primeras cinco filas del conjunto de datos?


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26



########################################################################################
¿Cuáles son las últimas cinco filas del conjunto de datos?


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15
99440,66dea50a8b16d9b4dee7af250b4be1a5,edb027a75a1449115f6b43211ae02a24,delivered,2018-03-08 20:57:30,2018-03-09 11:20:28,2018-03-09 22:11:59,2018-03-16 13:08:30,2018-04-03



########################################################################################
¿Cómo puedes obtener una muestra aleatoria de filas del conjunto de datos?


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
19963,2e320ab86de835aa42cf9e69f54f9380,6bb8f8549532be3e4fc1df6f7b281ed9,delivered,2017-11-18 19:50:31,2017-11-18 20:07:24,2017-11-22 15:42:21,2017-11-26 17:09:43,2017-12-11
52013,4c8962d1aff37c5cafe5190f4f9b6839,7f194dac460c33b8e5b69366f13b0d42,delivered,2018-01-09 22:43:11,2018-01-10 10:31:53,2018-01-10 17:53:37,2018-01-18 20:33:04,2018-02-08
72147,8e1ded141534096603108f1ee5c27735,3956d1e55818b7dc872d5a1de09e4ddc,delivered,2018-01-23 15:04:18,2018-01-23 15:15:32,2018-01-24 23:33:04,2018-01-30 19:35:28,2018-02-07
68964,93b885e8ae6403b9c59c24f1cd30e1ad,1f1d262df96204aa695fa4f0075ea33c,delivered,2017-05-11 21:52:00,2017-05-11 22:05:17,2017-05-12 12:43:20,2017-05-16 10:12:03,2017-05-22
92642,4d9029f2c1721a0d915d363ef3473499,6e498327113595d732931d56b625c876,delivered,2018-04-03 16:41:19,2018-04-03 17:28:49,2018-04-04 20:34:55,2018-04-09 19:38:46,2018-04-20



########################################################################################
¿Cuáles son las columnas del conjunto de datos?
	 - order_id
	 - customer_id
	 - order_status
	 - order_purchase_timestamp
	 - order_approved_at
	 - order_delivered_carrier_date
	 - order_delivered_customer_date
	 - order_estimated_delivery_date

########################################################################################
¿Cuál es el tipo de datos de cada columna?
order_id                                 object
customer_id                              object
order_status                             object
order_purchase_timestamp         datetime64[ns]
order_approved_at                datetime64[ns]
order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
dtype: object

########################################################################################
¿Cuántas columnas hay de cada tipo de datos

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
count,98666,98666,98666,98666,98652,97657,96476,98666
unique,98666,98666,7,,,,,
top,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,,,,,
freq,1,1,96478,,,,,
mean,,,,2017-12-31 22:05:22.366560,2018-01-01 09:30:11.667913216,2018-01-04 21:56:02.570537728,2018-01-14 12:09:19.035542272,2018-01-24 16:36:25.409360640
min,,,,2016-09-04 21:15:19,2016-09-15 12:16:38,2016-10-08 10:34:01,2016-10-11 13:46:32,2016-10-04 00:00:00
25%,,,,2017-09-12 23:21:27.500000,2017-09-13 15:37:19.500000,2017-09-15 22:31:51,2017-09-25 22:07:22.249999872,2017-10-04 00:00:00
50%,,,,2018-01-19 18:02:47,2018-01-20 09:10:39,2018-01-24 16:12:38,2018-02-02 19:28:10.500000,2018-02-15 00:00:00
75%,,,,2018-05-04 20:05:35.249999872,2018-05-05 07:16:04,2018-05-08 13:38:00,2018-05-15 22:48:52.249999872,2018-05-28 00:00:00
max,,,,2018-09-03 09:06:57,2018-09-03 17:40:06,2018-09-11 19:48:28,2018-10-17 13:22:46,2018-10-25 00:00:00



########################################################################################
¿Hay valores nulos en el conjunto de datos?
order_delivered_customer_date    2190
order_delivered_carrier_date     1009
order_approved_at                  14
order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_estimated_delivery_date       0
dtype: int64

########################################################################################
¿Cuál es el porcentaje de valores nulos en cada columna?
order_delivered_customer_date    2.22
order_delivered_carrier_date     1.02
order_approved_at                0.01
order_id                         0.00
customer_id                      0.00
order_status                     0.00
order_purchase_timestamp         0.00
order_estimated_delivery_date    0.00
dtype: float64

##################################################################################

In [ ]:
min_date_orders = df_orders['order_purchase_timestamp'].min()
max_date_orders = df_orders['order_purchase_timestamp'].max()

print(min_date_orders, max_date_orders)


2016-09-04 21:15:19 2018-09-03 09:06:57


# OLIST PRODUCTS

In [ ]:
df_products = pd.read_parquet('/content//drive/MyDrive/e-commerce TFM/Data/df_products.parquet')


In [ ]:
check_df(df=df_products, tipo="")


¿Cuántas filas y columnas hay en el conjunto de datos?
	Hay 32,341 filas y 9 columnas.

########################################################################################
¿Cuáles son las primeras cinco filas del conjunto de datos?


,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,Perfumería,40.0,287.0,1,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,Artes,44.0,276.0,1,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,Ocio deportivo,46.0,250.0,1,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,Bebes,27.0,261.0,1,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,Utilidades domesticas,37.0,402.0,4,625.0,20.0,17.0,13.0



########################################################################################
¿Cuáles son las últimas cinco filas del conjunto de datos?


,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
32946,a0b7d5a992ccda646f2d34e418fff5a0,Muebles de decoracion,45.0,67.0,2,12300.0,40.0,40.0,40.0
32947,bf4538d88321d0fd4412a93c974510e6,Iluminación de herramientas de construcción,41.0,971.0,1,1700.0,16.0,19.0,16.0
32948,9a7c6041fa9592d9d9ef6cfe62a71f8c,Baño mesa de cama,50.0,799.0,1,1400.0,27.0,7.0,27.0
32949,83808703fc0706a22e264b9d75f04a2e,Accesorios de computadora,60.0,156.0,2,700.0,31.0,13.0,20.0
32950,106392145fca363410d287a815be6de4,Baño mesa de cama,58.0,309.0,1,2083.0,12.0,2.0,7.0



########################################################################################
¿Cómo puedes obtener una muestra aleatoria de filas del conjunto de datos?


,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
971,68db47d312d0b87e469762f0481d3b9c,Herramientas de jardín,60.0,681.0,4,16900.0,38.0,70.0,38.0
2581,711027a82d762aad9b42dc032629dd18,Utilidades domesticas,53.0,1147.0,2,5650.0,35.0,22.0,30.0
3984,cf5f0f6ba7c1477ba6aca74da24faba5,Bebes,53.0,1601.0,5,29250.0,42.0,58.0,60.0
8125,9269d6409ec9f19cad883b2c1aa206d9,Utilidades domesticas,43.0,731.0,1,9750.0,42.0,28.0,33.0
18579,ac43429cfe2a3933709b85dc2acacca8,Regalos relojes,56.0,1075.0,4,650.0,18.0,14.0,16.0



########################################################################################
¿Cuáles son las columnas del conjunto de datos?
	 - product_id
	 - product_category_name
	 - product_name_lenght
	 - product_description_lenght
	 - product_photos_qty
	 - product_weight_g
	 - product_length_cm
	 - product_height_cm
	 - product_width_cm

########################################################################################
¿Cuál es el tipo de datos de cada columna?
product_id                     object
product_category_name          object
product_name_lenght           float64
product_description_lenght    float64
product_photos_qty              int64
product_weight_g              float64
product_length_cm             float64
product_height_cm             float64
product_width_cm              float64
dtype: object

########################################################################################
¿Cuántas columnas hay de cada tipo de datos?
float64    6
object     2
int64  

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
count,32341,32341,32341.0,32341.0,32341.0,32340.0,32340.0,32340.0,32340.0
unique,32341,73,,,,,,,
top,1e9e8ef04dbcff4541ed26657ea517e5,Baño mesa de cama,,,,,,,
freq,1,3029,,,,,,,
mean,,,48.476949,771.495285,2.188986,2276.956586,30.854545,16.958813,23.208596
std,,,10.245741,635.115225,1.736766,4279.291845,16.955965,13.636115,12.078762
min,,,5.0,4.0,1.0,0.0,7.0,2.0,6.0
25%,,,42.0,339.0,1.0,300.0,18.0,8.0,15.0
50%,,,51.0,595.0,1.0,700.0,25.0,13.0,20.0
75%,,,57.0,972.0,3.0,1900.0,38.0,21.0,30.0



########################################################################################
¿Hay valores nulos en el conjunto de datos?
product_weight_g              1
product_length_cm             1
product_height_cm             1
product_width_cm              1
product_id                    0
product_category_name         0
product_name_lenght           0
product_description_lenght    0
product_photos_qty            0
dtype: int64

########################################################################################
¿Cuál es el porcentaje de valores nulos en cada columna?
product_id                    0.0
product_category_name         0.0
product_name_lenght           0.0
product_description_lenght    0.0
product_photos_qty            0.0
product_weight_g              0.0
product_length_cm             0.0
product_height_cm             0.0
product_width_cm              0.0
dtype: float64

########################################################################################


In [ ]:
df_products['product_photos_qty'].unique()

array([ 1,  4,  2,  3,  5,  9,  6,  7, 12, 10, 11, 17,  8, 15, 13, 14, 20,
       18, 19])

In [ ]:
# Transformar la columna product_photos_qty a tipo entero
df_products['product_photos_qty'] = df_products['product_photos_qty'].astype(int)

# Verificar el cambio
print(df_products['product_photos_qty'].dtype)


int64


# OLIST SELLERS

In [ ]:
df_sellers = pd.read_parquet('/content//drive/MyDrive/e-commerce TFM/Data/df_sellers.parquet')


In [ ]:
check_df(df=df_sellers, tipo="")

¿Cuántas filas y columnas hay en el conjunto de datos?
	Hay 3,095 filas y 4 columnas.

########################################################################################
¿Cuáles son las primeras cinco filas del conjunto de datos?


,seller_id,seller_zip_code_prefix,seller_state,seller_city_normalized
0,3442f8959a84dea7ee197c632cb2df15,13023,SP,campinas
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,SP,mogi guacu
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,RJ,rio de janeiro
3,c0f3eea2e14555b6faeea3dd58c1b1c3,04195,SP,sao paulo
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,SP,braganca paulista



########################################################################################
¿Cuáles son las últimas cinco filas del conjunto de datos?


,seller_id,seller_zip_code_prefix,seller_state,seller_city_normalized
3090,98dddbc4601dd4443ca174359b237166,87111,PR,sarandi
3091,f8201cab383e484733266d1906e2fdfa,88137,SC,palhoca
3092,74871d19219c7d518d0090283e03c137,04650,SP,sao paulo
3093,e603cf3fec55f8697c9059638d6c8eb5,96080,RS,pelotas
3094,9e25199f6ef7e7c347120ff175652c3b,12051,SP,taubate



########################################################################################
¿Cómo puedes obtener una muestra aleatoria de filas del conjunto de datos?


,seller_id,seller_zip_code_prefix,seller_state,seller_city_normalized
2914,808d4348b916efa08e766ebad39f61eb,02336,SP,sao paulo
2518,e58a5b390e28abc0b216cfb0e07d27d7,22640,RJ,rio de janeiro
1556,f3295428338a40977a03f555246a70f4,29032,ES,vitoria
3052,9d5a9018aee56acb367ba9c3f05d1d6a,74930,GO,aparecida de goiania
2492,2235cf78d3e2073a88146a9cb60f0dd0,04547,SP,sao paulo



########################################################################################
¿Cuáles son las columnas del conjunto de datos?
	 - seller_id
	 - seller_zip_code_prefix
	 - seller_state
	 - seller_city_normalized

########################################################################################
¿Cuál es el tipo de datos de cada columna?
seller_id                 object
seller_zip_code_prefix    object
seller_state              object
seller_city_normalized    object
dtype: object

########################################################################################
¿Cuántas columnas hay de cada tipo de datos?
object    4
Name: count, dtype: int64

########################################################################################
¿Cómo podríamos obtener información más completa sobre la estructura y el contenido del DataFrame?
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 4 columns):
 #   Column                  N

,seller_id,seller_zip_code_prefix,seller_state,seller_city_normalized
count,3095,3095,3095,3095
unique,3095,2246,23,609
top,3442f8959a84dea7ee197c632cb2df15,14940,SP,sao paulo
freq,1,49,1849,695



########################################################################################
¿Hay valores nulos en el conjunto de datos?
seller_id                 0
seller_zip_code_prefix    0
seller_state              0
seller_city_normalized    0
dtype: int64

########################################################################################
¿Cuál es el porcentaje de valores nulos en cada columna?
seller_id                 0.0
seller_zip_code_prefix    0.0
seller_state              0.0
seller_city_normalized    0.0
dtype: float64

########################################################################################


# OLIST PRODUCT CATEGORY NAME TRANSLATION

In [ ]:
df_category_name_translation = pd.read_parquet('/content//drive/MyDrive/e-commerce TFM/Data/df_product_category_name_translation.parquet')


In [ ]:
check_df(df=df_category_name_translation, tipo="")


¿Cuántas filas y columnas hay en el conjunto de datos?
	Hay 71 filas y 2 columnas.

########################################################################################
¿Cuáles son las primeras cinco filas del conjunto de datos?


,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor



########################################################################################
¿Cuáles son las últimas cinco filas del conjunto de datos?


,product_category_name,product_category_name_english
66,flores,flowers
67,artes_e_artesanato,arts_and_craftmanship
68,fraldas_higiene,diapers_and_hygiene
69,fashion_roupa_infanto_juvenil,fashion_childrens_clothes
70,seguros_e_servicos,security_and_services



########################################################################################
¿Cómo puedes obtener una muestra aleatoria de filas del conjunto de datos?


,product_category_name,product_category_name_english
45,alimentos,food
9,relogios_presentes,watches_gifts
31,market_place,market_place
66,flores,flowers
3,cama_mesa_banho,bed_bath_table



########################################################################################
¿Cuáles son las columnas del conjunto de datos?
	 - product_category_name
	 - product_category_name_english

########################################################################################
¿Cuál es el tipo de datos de cada columna?
product_category_name            object
product_category_name_english    object
dtype: object

########################################################################################
¿Cuántas columnas hay de cada tipo de datos?
object    2
Name: count, dtype: int64

########################################################################################
¿Cómo podríamos obtener información más completa sobre la estructura y el contenido del DataFrame?
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 2 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  

,product_category_name,product_category_name_english
count,71,71
unique,71,71
top,beleza_saude,health_beauty
freq,1,1



########################################################################################
¿Hay valores nulos en el conjunto de datos?
product_category_name            0
product_category_name_english    0
dtype: int64

########################################################################################
¿Cuál es el porcentaje de valores nulos en cada columna?
product_category_name            0.0
product_category_name_english    0.0
dtype: float64

########################################################################################
